In [ ]:
d

## Import Required Libraries

In [1]:
import glob
import numpy as np
import pandas as pd

In [2]:
EXPERIMENT_ID = '1005'

experiment_dir = 'outputs/experiment_{}/'.format(EXPERIMENT_ID)
results_path = experiment_dir + 'results.npy'
schedules_path = experiment_dir + 'schedulesWithMakespan.npy'
log_paths = glob.glob(experiment_dir + '*.log')

## Read log files, and results

In [3]:
# logs = pd.concat([pd.read_csv(log_file) for log_file in log_paths], axis=0)
result_columns = ['batch_id', 'makespan_mean', 'cum_makespan_mean', 'time', 'cum_time']
results = pd.DataFrame(np.load(results_path), columns=result_columns)
with open(schedules_path, 'rb') as f:
    batch_schedules = np.load(f)
    batch_makespans = np.load(f)

In [7]:
def convert_schedules_np_to_df(batch_schedules, batch_makespans):
    """Convert batch schedules from numpy array to dataframe"""
    n_batch_instance, batch_size, n_nodes, _ = batch_schedules.shape
    schedules = batch_schedules.reshape((n_batch_instance*batch_size, n_nodes, 2))
    # remove machine-to-machine pair
    schedules = schedules[
        np.not_equal(
            schedules[:,:,:1], schedules[:,:,1:]
        )[:,:,-1]
    ].reshape(schedules.shape[0], -1, schedules.shape[-1]).astype(str)
    
    n_jobs = schedules.shape[1]
    makespans = batch_makespans.reshape((n_batch_instance*batch_size,1))

    df_schedules = pd.DataFrame(
        np.concatenate([
            np.apply_along_axis(
                ' -> '.join, 0, [schedules[:,:,0], schedules[:,:,1]]),
            makespans
        ], axis=1),
        columns= list(map(lambda v: "J_"+str(v), range(n_jobs)))+['makespan']
       )
    
    return df_schedules

In [8]:
df_schedules = convert_schedules_np_to_df(batch_schedules, batch_makespans)
df_schedules.head(10)

,J_0,J_1,J_2,J_3,J_4,makespan
0,4 -> 5,3 -> 5,0 -> 5,2 -> 6,1 -> 6,2.3685446
1,2 -> 5,3 -> 5,0 -> 5,4 -> 6,1 -> 6,2.329349
2,1 -> 5,2 -> 5,0 -> 5,4 -> 6,3 -> 6,2.0618036
3,3 -> 5,4 -> 5,2 -> 5,1 -> 6,0 -> 6,1.5268161
4,1 -> 5,0 -> 5,4 -> 6,3 -> 6,2 -> 6,1.7808145
5,3 -> 5,1 -> 5,0 -> 5,4 -> 6,2 -> 6,2.253179
6,4 -> 5,3 -> 5,0 -> 5,2 -> 6,1 -> 6,1.4486477
7,2 -> 5,1 -> 5,4 -> 6,0 -> 6,3 -> 6,1.6840694
8,3 -> 5,2 -> 5,1 -> 6,4 -> 6,0 -> 6,1.0499716
9,3 -> 5,2 -> 5,4 -> 6,0 -> 6,1 -> 6,1.8646028


In [28]:
makespans.shape,  schedules.shape
np.apply_along_axis(
        ' -> '.join, 0, [schedules[:,:,0], schedules[:,:,1]]).shape

np.concatenate([
    np.apply_along_axis(
        ' -> '.join, 0, [schedules[:,:,0], schedules[:,:,1]]),
    makespans
], axis=1)


array([['5 -> 5', '3 -> 5', '0 -> 5', '1 -> 5', '2 -> 5', '6 -> 6',
        '4 -> 6', '4.1202216'],
       ['5 -> 5', '0 -> 5', '1 -> 5', '6 -> 6', '3 -> 6', '4 -> 6',
        '2 -> 6', '1.8093971'],
       ['5 -> 5', '1 -> 5', '4 -> 5', '3 -> 5', '6 -> 6', '0 -> 6',
        '2 -> 6', '2.050342'],
       ['5 -> 5', '0 -> 5', '3 -> 5', '4 -> 5', '6 -> 6', '1 -> 6',
        '2 -> 6', '3.4389796'],
       ['5 -> 5', '2 -> 5', '0 -> 5', '1 -> 5', '4 -> 5', '3 -> 5',
        '6 -> 6', '4.6745625'],
       ['5 -> 5', '1 -> 5', '6 -> 6', '3 -> 6', '4 -> 6', '0 -> 6',
        '2 -> 6', '3.0054615'],
       ['5 -> 5', '2 -> 5', '3 -> 5', '0 -> 5', '1 -> 5', '4 -> 5',
        '6 -> 6', '5.427072'],
       ['5 -> 5', '1 -> 5', '3 -> 5', '2 -> 5', '4 -> 5', '6 -> 6',
        '0 -> 6', '3.291935'],
       ['5 -> 5', '4 -> 5', '3 -> 5', '0 -> 5', '2 -> 5', '1 -> 5',
        '6 -> 6', '6.268305'],
       ['5 -> 5', '1 -> 5', '6 -> 6', '3 -> 6', '0 -> 6', '2 -> 6',
        '4 -> 6', '3.5240579'],
    

In [41]:
#"V_".join(list(range(n_nodes)))

list(map(lambda v: "V_"+str(v), range(1,n_nodes+1)))+['makespan']

['V_1', 'V_2', 'V_3', 'V_4', 'V_5', 'V_6', 'V_7', 'makespan']

In [38]:
"V_".join(map(str, range(1, n_nodes+1)))

'1V_2V_3V_4V_5V_6V_7'

,V_0,V_1,V_2,V_3,V_4,V_5,V_6,makespan
0,5 -> 5,3 -> 5,0 -> 5,1 -> 5,2 -> 5,6 -> 6,4 -> 6,4.1202216
1,5 -> 5,0 -> 5,1 -> 5,6 -> 6,3 -> 6,4 -> 6,2 -> 6,1.8093971
2,5 -> 5,1 -> 5,4 -> 5,3 -> 5,6 -> 6,0 -> 6,2 -> 6,2.050342
3,5 -> 5,0 -> 5,3 -> 5,4 -> 5,6 -> 6,1 -> 6,2 -> 6,3.4389796
4,5 -> 5,2 -> 5,0 -> 5,1 -> 5,4 -> 5,3 -> 5,6 -> 6,4.6745625
...,...,...,...,...,...,...,...,...
59,5 -> 5,6 -> 6,0 -> 6,4 -> 6,2 -> 6,3 -> 6,1 -> 6,1.8668945
60,5 -> 5,3 -> 5,2 -> 5,6 -> 6,0 -> 6,1 -> 6,4 -> 6,2.3066711
61,5 -> 5,2 -> 5,4 -> 5,1 -> 5,6 -> 6,3 -> 6,0 -> 6,2.3086023
62,5 -> 5,2 -> 5,3 -> 5,0 -> 5,6 -> 6,1 -> 6,4 -> 6,2.3543978


In [9]:
makespans.shape

(64,)

In [106]:
batch_makespans

array([[[4.1202216],
        [1.8093971],
        [2.050342 ],
        [3.4389796]],

       [[4.6745625],
        [3.0054615],
        [5.427072 ],
        [3.291935 ]],

       [[6.268305 ],
        [3.5240579],
        [2.022379 ],
        [2.9663215]],

       [[2.97403  ],
        [1.9081707],
        [1.6280885],
        [3.7251925]],

       [[4.695125 ],
        [4.4765887],
        [3.2056866],
        [3.1705523]],

       [[3.486442 ],
        [3.0442905],
        [2.5101833],
        [2.1588836]],

       [[2.9759874],
        [1.4730709],
        [2.977961 ],
        [2.5275483]],

       [[2.4707196],
        [2.7855203],
        [3.2157311],
        [2.393405 ]],

       [[2.962733 ],
        [4.4704065],
        [3.3364108],
        [1.8417923]],

       [[3.9044058],
        [2.5286293],
        [2.6631856],
        [3.4287446]],

       [[4.5789757],
        [2.829761 ],
        [3.2408855],
        [2.1371822]],

       [[3.107292 ],
        [3.1559463],
        [4.6

In [103]:
# r = best_schedules.reshape((64, 7, 2))
# #np.transpose(r, (0,2,1))
# r = r.astype(str)

# r[:,:,0]#  + "," + r[:,:,1] 

# #np.core.defchararray.add(r[:,:,0], r[:,:,1])

# # list(zip(r[:,:,0], r[:,:,1]))

# # numpy.char.join(r[:,:,0], r[:,:,1])








,0,1,2,3,4,5,6
0,5 -> 5,3 -> 5,0 -> 5,1 -> 5,2 -> 5,6 -> 6,4 -> 6
1,5 -> 5,0 -> 5,1 -> 5,6 -> 6,3 -> 6,4 -> 6,2 -> 6
2,5 -> 5,1 -> 5,4 -> 5,3 -> 5,6 -> 6,0 -> 6,2 -> 6
3,5 -> 5,0 -> 5,3 -> 5,4 -> 5,6 -> 6,1 -> 6,2 -> 6
4,5 -> 5,2 -> 5,0 -> 5,1 -> 5,4 -> 5,3 -> 5,6 -> 6
...,...,...,...,...,...,...,...
59,5 -> 5,6 -> 6,0 -> 6,4 -> 6,2 -> 6,3 -> 6,1 -> 6
60,5 -> 5,3 -> 5,2 -> 5,6 -> 6,0 -> 6,1 -> 6,4 -> 6
61,5 -> 5,2 -> 5,4 -> 5,1 -> 5,6 -> 6,3 -> 6,0 -> 6
62,5 -> 5,2 -> 5,3 -> 5,0 -> 5,6 -> 6,1 -> 6,4 -> 6


In [53]:
#best_makespans.shape, 

a = np.arange(6).reshape((3, 2))

In [ ]:
np.chararray((3, 3))

In [3]:

path = 'outputs/experiment_{}/schedulesWithMakespan.npy'.format(experiment_id)

with open(path, 'rb') as f:
    best_schedules = np.load(f)
    best_makespans = np.load(f)

In [4]:
path = 'outputs/experiment_{}/results.npy'.format(experiment_id)
with open(path, 'rb') as f:
    results = np.load(f)

In [ ]:
log_path = 'outputs/experiment_{}/results.npy'.format(experiment_id)

In [5]:
_columns = ['batch_id', 'makespan_mean', 'cum_makespan_mean', 'time', 'cum_time']
df = pd.DataFrame(results, columns=_columns)
df

,batch_id,makespan_mean,cum_makespan_mean,time,cum_time
0,0.0,3.189371,3.189371,0.016163,0.117919
1,1.0,3.750617,3.469994,0.017083,0.159562
2,2.0,2.911465,3.283818,0.016928,0.201338
3,3.0,2.761600,3.153264,0.016646,0.242393


# Modules: Datasets and Graphs

In [1]:
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras.layers import Layer

from msp.datasets import make_data, load_sample_data, make_sparse_data
from msp.graphs import MSPGraph, MSPSparseGraph
from msp.models.encoders import GGCNEncoder
from msp.models.decoders import AttentionDecoder

%load_ext autoreload
%autoreload 2

In [2]:
dataset = make_sparse_data(40, msp_size=(2,5), random_state=2021)

In [31]:
type_ = tf.constant([[[4,3,6,1]],[[9,3,6,1]]])

# tf.slice(type_, [0,0,2], [-1,-1,-1])
temp = tf.zeros(type_.shape)
cur_node = tf.constant([[2],[1]])
tf.tensor_scatter_nd_update(temp, indices, [1,1])



self._visited = tf.tensor_scatter_nd_update(
    tf.squeeze(temp, axis=-2),
    tf.concat([tf.reshape(tf.range(batch_size, dtype=tf.int64),cur_node.shape), cur_node], axis=1),
    tf.ones((batch_size,), dtype=self._visited.dtype)
)[:,tf.newaxis,:]


InvalidArgumentError: Inner dimensions of output shape must match inner dimensions of updates shape. Output: [2,1,4] updates: [2] [Op:TensorScatterUpdate]

In [16]:
indices = tf.constant([[2],[1]])
tf.gather(tf.squeeze(type_), indices, batch_dims=1)

<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[6],
       [3]], dtype=int32)>

In [15]:
tf.squeeze(type_)

TensorShape([2, 4])

In [3]:
batch_size = 2
batch_dataset = dataset.batch(batch_size)
inputs = list(batch_dataset.take(1))[0]
inputs

MSPSparseGraph(msp_size=<tf.Tensor: shape=(2, 2), dtype=uint8, numpy=
array([[2, 5],
       [2, 5]], dtype=uint8)>, adj_matrix=<tf.Tensor: shape=(2, 7, 7), dtype=uint8, numpy=
array([[[0, 1, 1, 0, 1, 1, 1],
        [1, 0, 1, 1, 1, 1, 1],
        [1, 1, 0, 1, 1, 1, 1],
        [0, 1, 1, 0, 1, 1, 1],
        [1, 1, 1, 1, 0, 1, 1],
        [1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 0, 0]],

       [[0, 1, 1, 1, 1, 1, 1],
        [1, 0, 1, 1, 1, 1, 1],
        [1, 1, 0, 1, 1, 1, 1],
        [1, 1, 1, 0, 1, 1, 1],
        [1, 1, 1, 1, 0, 1, 1],
        [1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 0, 0]]], dtype=uint8)>, node_features=<tf.Tensor: shape=(2, 7, 5), dtype=float64, numpy=
array([[[0.60597828, 0.73336936, 0.13894716, 0.31267308, 1.        ],
        [0.12816238, 0.17899311, 0.75292543, 0.66216051, 1.        ],
        [0.0968944 , 0.05857129, 0.96239599, 0.61655744, 1.        ],
        [0.56127236, 0.61652471, 0.96384302, 0.57430429, 1.        ],
        [0.45214524, 0.201

# Modules: Layers and Encoders

In [4]:
units = 6
layers = 2
ecoder_model = GGCNEncoder(units, layers)
embedded_inputs = ecoder_model(inputs)

In [5]:
embedded_inputs.node_embed

<tf.Tensor: shape=(2, 7, 6), dtype=float32, numpy=
array([[[-1.4382148e-01,  9.6293068e-01,  2.6263008e+00,  8.1934345e-01,
         -8.5672736e-02,  5.8940428e-01],
        [ 3.0298412e-02,  1.6358614e+00,  1.6347841e+00,  3.6575854e-02,
          1.5360398e+00,  1.0922415e+00],
        [ 1.7571726e-01,  1.7597905e+00,  1.7030218e+00, -6.0753770e-02,
          1.6991560e+00,  1.3841438e+00],
        [ 3.7202388e-02,  1.2748857e+00,  1.8815839e+00,  1.7817324e-01,
          6.8495989e-01,  9.9834001e-01],
        [ 3.0412453e-01,  5.2222323e-01,  1.6923902e+00,  5.4357314e-01,
          3.6024973e-01,  3.4970191e-01],
        [ 1.7666927e-01,  2.9611260e-01,  1.0662408e+00,  6.4383304e-01,
          2.4254315e-01,  0.0000000e+00],
        [ 1.7666927e-01,  2.9611260e-01,  1.0662408e+00,  6.4383304e-01,
          2.4254315e-01,  0.0000000e+00]],

       [[ 1.6468465e-03,  1.0810244e+00,  2.1540470e+00,  6.9013402e-02,
          1.8178177e-01,  8.9684927e-01],
        [-1.2560275e-01,  1

In [6]:
embedded_inputs.edge_embed

<tf.Tensor: shape=(2, 7, 7, 6), dtype=float32, numpy=
array([[[[ 2.50189126e-01,  1.01695740e+00,  6.23385191e-01,
           8.16367984e-01,  6.22225940e-01,  4.46099967e-01],
         [ 1.16019940e+00,  1.72448909e+00,  3.93749446e-01,
           3.10133249e-01, -5.01590371e-01,  5.63548148e-01],
         [ 1.43835652e+00,  2.24321032e+00,  5.32857537e-01,
          -7.28480518e-04, -1.19393364e-01,  1.77944869e-01],
         [ 7.98115373e-01,  1.17682934e+00,  8.40444684e-01,
           5.86094320e-01,  6.14124000e-01,  1.51605129e-01],
         [ 1.12041509e+00,  1.64951026e+00,  6.24807000e-01,
          -1.42291248e-01, -1.41600370e-01,  6.89730287e-01],
         [-3.71431917e-01,  4.45764840e-01,  9.94485259e-01,
           1.86788011e+00,  1.60849881e+00, -1.35840312e-01],
         [-3.71431917e-01,  4.45764840e-01,  9.94485259e-01,
           1.86788011e+00,  1.60849881e+00, -1.35840312e-01]],

        [[ 1.29513764e+00,  1.03694320e+00,  4.34905380e-01,
           1.72726357e

# Modules: Decoder

In [ ]:
model = AttentionDecoder(units, aggregation_graph='mean', n_heads=3)
makespan, ll, pi = model(embedded_inputs)
pi

# All together

In [ ]:
n_instances = 640
instance_size = (2, 8)
batch_size = 64
units = 64  # embedding dims
layers = 2

dataset = make_sparse_data(n_instances, msp_size=instance_size, random_state=2021)
batch_dataset = dataset.batch(batch_size)
inputs = list(batch_dataset.take(1))[0]

ecoder_model = GGCNEncoder(units, layers)
embedded_inputs = ecoder_model(inputs)

n_heads = 8
model = AttentionDecoder(units, aggregation_graph='mean', n_heads=n_heads)
makespan, ll, pi = model(embedded_inputs)
pi

In [ ]:
inputs.adj_matrix[0]


tf.math.reduce_all(tf.math.equal(inputs.adj_matrix[1], inputs.adj_matrix[2]))

In [ ]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

print(inputs.adj_matrix[0].numpy())

In [ ]:
print(inputs.adj_matrix[7].numpy())

In [ ]:
from typing import NamedTuple

from tensorflow import Tensor

class MSPState:

    def __init__(self, inputs):
        """ """
        self.adj_matrix = inputs.adj_matrix
        self.node_embed = inputs.node_embed
        
        batch_size, num_nodes, node_embed_dims = self.node_embed.shape
        
        self._first_node = tf.zeros((batch_size,1), dtype=tf.int64)
        self._last_node = self._first_node
        self._visited = tf.zeros((batch_size,1,num_nodes), dtype=tf.int64)
        self._makespan = tf.zeros((batch_size,1))

        self.i = tf.zeros(1, dtype=tf.int64) # # Vector with length num_steps
        self.ids = tf.range(5, delta=1, dtype=tf.int64)[:, None] #  # Add steps dimension
        #self._step_num = tf.zeros(1, dtype=tf.int64)


    @property
    def first_node(self):
        return self._first_node

    


    

        # self._node_embed = None
        # self._edge_embed = None
        # self._adj_matrix = None
        # self._first_job = None
        # self._last_job = None

    # @property
    # def node_embed(self):
    #     return self._node_embed

    # @node_embed.setter
    # def node_embed(self, node_embed):
    #     self._node_embed = node_embed

    # @property
    # def adj_matrix(self):
    #     return self._adj_matrix

    # @adj_matrix.setter
    # def adj_matrix(self, adj_matrix):
    #     self._adj_matrix = adj_matrix

    # def initialize(self, inputs):
    #     self.adj_matrix = inputs.adj_matrix
    #    return self




class MSPSolution:

    def __init__(self):
        pass

    # sequence: Tensor
    # makespan: Tensor
    # a: int

In [ ]:
import torch
torch.arange(5, dtype=torch.int64)[:, None], tf.range(5, delta=1, dtype=tf.int64)[:, None]

In [ ]:
class MSPSparseGraph2(NamedTuple):
    """ """
    adj_matrix: Tensor
    node_features: Tensor
    edge_features: Tensor
    alpha: Tensor

    @property
    def num_node(self):
        return self.adj_matrix.shape[-2]

In [ ]:
from collections import namedtuple

class A(NamedTuple):

    a: int

    @property
    def num(self):
        return self.a

    # jane._replace(a=26)


class C(
    NamedTuple(
        'SSSSS',
        A._field_types.items()
    ),
    A
):
    pass



class B(
    NamedTuple(
        'SSSB',
        [
            *A._field_types.items(),
            ('weight', int)
        ]
    ),
    A
):
    @property
    def h(self):
        return self.weight


# class B(namedtuple('SSSB', [*A._fields, "weight"]), A):
#     #[*A._fields, "weight"]

#     @property
#     def h(self):
#         return self.weight

    

    # def __getitem__(self, key):
    #     return self[key]

In [ ]:
#B(A(a=90)._asdict(), weight=90)
#B(**A(a=9090)._asdict(), weight=90).num
#Ba = 2343, weight=23).num
#dir(B)
#C(a=90)
B(*(34,), weight=34)

In [ ]:
list(A._field_types.items())

In [ ]:
A._field_types

In [ ]:
def fun(a):
    print(a)

fun(**inputs._asdict())


In [ ]:
[*A._field_types, ('v','d')]

In [ ]:
# for attr, val in inputs._asdict().items():
#     print(attr, val)

In [ ]:
from typing import NamedTuple
class AttentionModelFixed(NamedTuple):
    """
    Context for AttentionModel decoder that is fixed during decoding so can be precomputed/cached
    This class allows for efficient indexing of multiple Tensors at once
    """
    node_embeddings: torch.Tensor
    context_node_projected: torch.Tensor
    glimpse_key: torch.Tensor
    glimpse_val: torch.Tensor
    logit_key: torch.Tensor

    def __getitem__(self, key):
        if tf.is_tensor(key) or isinstance(key, slice):
            return AttentionModelFixed(
                node_embeddings=self.node_embeddings[key],
                context_node_projected=self.context_node_projected[key],
                glimpse_key=self.glimpse_key[:, key],  # dim 0 are the heads
                glimpse_val=self.glimpse_val[:, key],  # dim 0 are the heads
                logit_key=self.logit_key[key]
            )
        return super(AttentionModelFixed, self).__getitem__(key)

In [ ]:
"""

"""
import math

import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras import Model

from msp.layers import GGCNLayer
from msp.graphs import MSPSparseGraph
from msp.solutions import MSPState

class AttentionDecoder(Model):

    def __init__(self, 
                 units,
                 *args, 
                 activation='relu',
                 aggregation_graph='mean',
                 n_heads=2, # make it 8
                 mask_inner=True,
                 tanh_clipping=10,
                 decode_type='sampling',
                 extra_logging=False,
                 **kwargs):
        """ """
        super(AttentionDecoder, self).__init__(*args, **kwargs)
        self.aggregation_graph = aggregation_graph
        self.n_heads = n_heads
        self.mask_inner = mask_inner
        self.tanh_clipping = tanh_clipping
        self.decode_type = decode_type
        self.extra_logging = extra_logging

        embedding_dim = units
        
        self.W_placeholder = self.add_weight(shape=(2*embedding_dim,),
                                initializer='random_uniform', #Placeholder should be in range of activations (think)
                                name='W_placeholder',
                                trainable=True)

        graph_embed_shape = tf.TensorShape((None, units))
        self.fixed_context_layer = tf.keras.layers.Dense(units, use_bias=False)
        self.fixed_context_layer.build(graph_embed_shape)

        # For each node we compute (glimpse key, glimpse value, logit key) so 3 * embedding_dim
        project_node_embeddings_shape = tf.TensorShape((None, None, None, units))
        self.project_node_embeddings = tf.keras.layers.Dense(3*units, use_bias=False)
        self.project_node_embeddings.build(project_node_embeddings_shape)

        #
        # Embedding of first and last node
        step_context_dim = 2*units
        project_step_context_shape = tf.TensorShape((None, None, step_context_dim))
        self.project_step_context = tf.keras.layers.Dense(embedding_dim, use_bias=False)
        self.project_step_context.build(project_step_context_shape)

        assert embedding_dim % n_heads == 0
        # Note n_heads * val_dim == embedding_dim so input to project_out is embedding_dim

        project_out_shape = tf.TensorShape((None, None, 1, embedding_dim))
        self.project_out = tf.keras.layers.Dense(embedding_dim, use_bias=False)
        self.project_out.build(project_out_shape)


        # self.context_layer = tf.keras.layers.Dense(units, use_bias=False)
        # self.mha_layer = None
        

        # dynamic router


        
    def call(self, inputs, training=False, return_pi=False):
        """ """
        state = MSPState(inputs)

        node_embedding = inputs.node_embed

        # AttentionModelFixed(node_embedding, fixed_context, *fixed_attention_node_data)
        fixed = self._precompute(node_embedding)


        # for i in range(num_steps):
            # i == 0 should be machine 
            # AttentionCell(inputs, states)

        outputs = []
        sequences = []

        # i = 0
        while not state.all_finished():
            # B x 1 x V
            # Get log probabilities of next action
            log_p, mask = self._get_log_p(fixed, state)
            
            selected = self._select_node(
                    tf.squeeze(tf.exp(log_p), axis=-2), tf.squeeze(mask, axis=-2)) # Squeeze out steps dimension

            state.update(selected)

            outputs.append(log_p[:, 0, :])
            sequences.append(selected)
            
            # if i == 1:
            #     break
            # i+=1
        
        _log_p, pi = tf.stack(outputs, axis=1), tf.stack(sequences, axis=1)

        if self.extra_logging:
            self.log_p_batch = _log_p
            self.log_p_sel_batch = tf.gather(tf.squeeze(_log_p,axis=-2), pi, batch_dims=1)

        # # Get predicted costs
        # cost, mask = self.problem.get_costs(nodes, pi)
        mask = None

        ###################################################
        # Need Clarity #############################################################
        # loglikelihood 
        ll = self._calc_log_likelihood(_log_p, pi, mask)

        ## Just for checking
        return_pi = True    
        if return_pi:
            return state.makespan, ll, pi

        return state.makespan, ll

        

        



    def _precompute(self, node_embedding, num_steps=1):

        graph_embed = self._get_graph_embed(node_embedding)

        fixed_context = self.fixed_context_layer(graph_embed)
        # fixed context = (batch_size, 1, embed_dim) to make broadcastable with parallel timesteps
        fixed_context = tf.expand_dims(fixed_context, axis=-2)

        glimpse_key_fixed, glimpse_val_fixed, logit_key_fixed  = tf.split(
            self.project_node_embeddings(tf.expand_dims(node_embedding, axis=-3)),
            num_or_size_splits=3,
            axis=-1
        )

        # No need to rearrange key for logit as there is a single head
        fixed_attention_node_data = (
            self._make_heads(glimpse_key_fixed, num_steps),
            self._make_heads(glimpse_val_fixed, num_steps),
            logit_key_fixed
        )
        return AttentionModelFixed(node_embedding, fixed_context, *fixed_attention_node_data)

    def _get_graph_embed(self, node_embedding):
        """ """
        if self.aggregation_graph == "sum":
            graph_embed = tf.reduce_sum(node_embedding, axis=-2)
        elif self.aggregation_graph == "max":
            graph_embed = tf.reduce_max(node_embedding, axis=-2)
        elif self.aggregation_graph == "mean":
            graph_embed = tf.reduce_mean(node_embedding, axis=-2)
        else:  # Default: dissable graph embedding
            graph_embed = tf.reduce_sum(node_embedding, axis=-2) * 0.0

        return graph_embed

    def _make_heads(self, v, num_steps=None):

        assert num_steps is None or v.shape[1] == 1 or v.shape[1] == num_steps
        batch_size, _, num_nodes, embed_dims = v.shape
        num_steps = num_steps if num_steps else 1
        head_dims = embed_dims//self.n_heads

        # M x B x N x V x (H/M)
        return tf.transpose(
            tf.broadcast_to(
                tf.reshape(v, shape=[batch_size, v.shape[1], num_nodes, self.n_heads, head_dims]),
                shape=[batch_size, num_steps, num_nodes, self.n_heads, head_dims]
            ),
            perm=[3, 0, 1, 2, 4]
        )

    def _get_log_p(self, fixed, state, normalize=True):
        # Compute query = context node embedding
        
        # B x 1 x H
        query = fixed.context_node_projected + \
                self.project_step_context(self._get_parallel_step_context(fixed.node_embeddings, state))
        
        # Compute keys and values for the nodes
        glimpse_K, glimpse_V, logit_K = self._get_attention_node_data(fixed, state)

        # Compute the mask, for masking next action based on previous actions
        mask = state.get_mask()
        graph_mask = state.get_graph_mask()

        # Compute logits (unnormalized log_p)
        log_p, glimpse = self._one_to_many_logits(query, glimpse_K, glimpse_V, logit_K, mask, graph_mask)

        # [B x N x V]
        # log-softmax activation function so that we get log probabilities over actions
        if normalize:
            log_p = tf.nn.log_softmax(log_p/1.0, axis=-1)

        assert not tf.reduce_any(tf.math.is_nan(log_p)), "Log probabilities over the nodes should be defined"

        return log_p, mask


    def _get_parallel_step_context(self, node_embedding, state, from_depot=False):
        """
        Returns the context per step, optionally for multiple steps at once 
        (for efficient evaluation of the model)
        """
        # last_node at time t
        last_node = state.get_current_node()
        batch_size, num_steps = last_node.shape

        if num_steps == 1:  # We need to special case if we have only 1 step, may be the first or not
            if state.i.numpy()[0] == 0:
                # First and only step, ignore prev_a (this is a placeholder)
                # B x 1 x 2H
                return tf.broadcast_to(self.W_placeholder[None, None, :], 
                                       shape=[batch_size, 1, self.W_placeholder.shape[-1]])
            else:
                return tf.concat(
                    [
                        tf.gather(node_embedding,state.first_node,batch_dims=1), 
                        tf.gather(node_embedding,last_node,batch_dims=1)  
                    ],
                    axis=-1
                )
                
                # print('$'*20)
                # node_embedding = torch.from_numpy(node_embedding.numpy())
                # f = torch.from_numpy(state.first_node.numpy())
                # l = torch.from_numpy(state.last_node.numpy())
                # GG = node_embedding\
                # .gather(
                #     1,
                #     torch.cat((f, l), 1)[:, :, None].expand(batch_size, 2, node_embedding.size(-1))
                # ).view(batch_size, 1, -1)
                # print(GG)
                # print('$'*20)
                # ##############################################
                # # PENDING
                # ##############################################
                # pass

    def _get_attention_node_data(self, fixed, state):
        return fixed.glimpse_key, fixed.glimpse_val, fixed.logit_key

    def _one_to_many_logits(self, query, glimpse_K, glimpse_V, logit_K, mask, graph_mask=None):
        batch_size, num_steps, embed_dim = query.shape
        query_size = key_size = val_size = embed_dim // self.n_heads

        # M x B x N x 1 x (H/M)
        # Compute the glimpse, rearrange dimensions to (n_heads, batch_size, num_steps, 1, key_size)
        glimpse_Q = tf.transpose(
            tf.reshape(
                query, # B x 1 x H
                shape=[batch_size, num_steps, self.n_heads, 1, query_size]
            ),
            perm=[2, 0, 1, 3, 4]
        )

        # [M x B x N x 1 x (H/M)] X [M x B x N x (H/M) x V] = [M x B x N x 1 x V]
        # Batch matrix multiplication to compute compatibilities (n_heads, batch_size, num_steps, graph_size)
        compatibility = tf.matmul(glimpse_Q, tf.transpose(glimpse_K, [0,1,2,4,3])) / math.sqrt(query_size)
        
        mask_temp = tf.cast(tf.broadcast_to(mask[None, :, :, None, :], shape=compatibility.shape), dtype=tf.double)
        compatibility = tf.cast(compatibility, dtype=tf.double) + (mask_temp * -1e9)

        graph_mask_temp = tf.cast(tf.broadcast_to(graph_mask[None, :, :, None, :], shape=compatibility.shape), dtype=tf.double)
        compatibility = tf.cast(compatibility, dtype=tf.double) + (graph_mask_temp * -1e9)

        compatibility = tf.cast(compatibility, dtype=tf.float32)        

        # compatibility[tf.broadcast_to(mask[None, :, :, None, :], shape=compatibility.shape)] = -1e10
        # compatibility[tf.broadcast_to(graph_mask[None, :, :, None, :], shape=compatibility.shape)] = -1e10

        # attention weights a(c,j): 
        attention_weights = tf.nn.softmax(compatibility, axis=-1)


        # [M x B x N x 1 x V] x [M x B x N x V x (H/M)] = [M x B x N x 1 x (H/M)]
        heads = tf.matmul(attention_weights, glimpse_V)
       
        # B x N x 1 x H
        # Project to get glimpse/updated context node embedding (batch_size, num_steps, embedding_dim)
        glimpse = self.project_out(
            tf.reshape(
                tf.transpose(heads, perm=[1, 2, 3, 0, 4]),
                shape=[batch_size, num_steps, 1, self.n_heads*val_size]
            )
        )

        # B x N x 1 x H
        # Now projecting the glimpse is not needed since this can be absorbed into project_out
        # final_Q = self.project_glimpse(glimpse)
        final_Q = glimpse


        # [B x N x 1 x H] x [B x 1 x H x V] = [B x N x 1 x V] --> [B x N x V] (Squeeze) 
        # Batch matrix multiplication to compute logits (batch_size, num_steps, graph_size)
        # logits = 'compatibility'
        logits = tf.squeeze(tf.matmul(final_Q, tf.transpose(logit_K, perm=[0,1,3,2])),
                            axis=-2) / math.sqrt(final_Q.shape[-1])

        logits = logits + ( tf.cast(graph_mask, dtype=tf.float32) * -1e9)
        logits = tf.math.tanh(logits) * self.tanh_clipping
        logits = logits + ( tf.cast(mask, dtype=tf.float32) * -1e9)

        # logits[graph_mask] = -1e10 
        # logits = torch.tanh(logits) * self.tanh_clipping
        # logits[mask] = -1e10
        
        return logits, tf.squeeze(glimpse, axis=-2)

    
    def _select_node(self, probs, mask):
        assert tf.reduce_all(probs == probs) == True, "Probs should not contain any nans"

        if self.decode_type == "greedy":
            selected = tf.math.argmax(probs, axis=1)
            assert not tf.reduce_any(tf.cast(tf.gather_nd(mask, tf.expand_dims(selected, axis=-1), batch_dims=1), dtype=tf.bool)), "Decode greedy: infeasible action has maximum probability"

        elif self.decode_type == "sampling":
            dist = tfp.distributions.Multinomial(total_count=1, probs=probs)
            selected = tf.argmax(dist.sample(), axis=1)

            # Check if sampling went OK
            while tf.reduce_any(tf.cast(tf.gather_nd(mask, tf.expand_dims(selected, axis=-1), batch_dims=1), dtype=tf.bool)):
                print('Sampled bad values, resampling!')
                selected = tf.argmax(dist.sample(), axis=1)

        else:
            assert False, "Unknown decode type"
        return selected

    
    def _calc_log_likelihood(self, _log_p, a, mask):
        
        # Get log_p corresponding to selected actions
        batch_size, steps_count = a.shape
        indices = tf.concat([
        tf.expand_dims(tf.broadcast_to(tf.range(steps_count, dtype=tf.int64), shape=a.shape), axis=-1),
        tf.expand_dims(a, axis=-1)],
        axis=-1
        )
        log_p = tf.gather_nd(_log_p, indices, batch_dims=1)
        

        # _log_p = torch.from_numpy(_log_p.numpy())
        # a = torch.from_numpy(a.numpy())
        # AA = _log_p.gather(2, a.unsqueeze(-1)).squeeze(-1)
        # print(AA)
        # print(log_p)
        # print('DONE')
        

        # Get log_p corresponding to selected actions
        # log_p = tf.gather(tf.squeeze(_log_p,axis=-2), a, batch_dims=1) #_log_p.gather(2, a.unsqueeze(-1)).squeeze(-1)

        # Optional: mask out actions irrelevant to objective so they do not get reinforced
        if mask is not None:
            log_p[mask] = 0

        # Why??????
        # assert (log_p > -1000).data.all(), "Logprobs should not be -inf, check sampling procedure!"

        # Calculate log_likelihood
        return tf.reduce_sum(log_p, axis=1) # log_p.sum(1)

       



In [ ]:
# import numpy as np

In [ ]:
# # x = tf.constant([[True,  True], [False, False]])
# # assert not tf.reduce_any(x), "asdf"
# tf.keras.losses.SparseCategoricalCrossentropy(
#     from_logits=True, reduction='none')([]).dtype

In [ ]:
# t = torch.tensor([[[2,3],[5,2]], [[6,1],[0,4]]])
# indices = torch.tensor([[[0, 0]],[[0, 0]]])
# torch.gather(t, 1, indices)
 
# # t = tf.constant([[[2,3],[5,2]], [[6,1],[0,4]]])
# # indices = tf.constant([[0,0,0]])
# # tf.gather_nd(t, indices=indices).numpy()

In [ ]:
# # #tf.gather_nd()
# # t = tf.constant([[[1,0],[1,0]], [[0,1],[1,1]]])
# # t = embedded_inputs.adj_matrix
# # indices = tf.constant([[0], [0]])
# # indices = tf.expand_dims(indices, axis=-1)

# # ~tf.cast(tf.gather_nd(t, indices=indices, batch_dims=1), tf.bool)
# a = tf.constant([-3.0,-1.0, 0.0,1.0,3.0], dtype = tf.float32)
# b = tf.keras.activations.tanh(a) 
# c = tf.math.tanh(a)
# b.numpy(), c.numpy()


In [ ]:
model = AttentionDecoder(6, aggregation_graph='mean', n_heads=3)
makespan, ll, pi = model(embedded_inputs)
pi

In [ ]:
tf.range(4, dtype=tf.int64)

In [ ]:

indices = [[[1, 0]], [[0, 1]]]
params = [[['a0', 'b0'], ['c0', 'd0']],
            [['a1', 'b1'], ['c1', 'd1']]]
#output = [['c0'], ['b1']]
tf.gather

In [ ]:
a = tf.Tensor(
[[[0,1]
  [1,2]],

 [[0,0]
#   [1 1]]], shape=(2, 2, 2), dtype=int64)
# tf.Tensor(
# [[[-1.0886807e+00 -1.2036482e+00 -1.0126851e+00]
#   [-9.8199100e+00 -1.0000000e+09 -5.4357959e-05]]

#  [[-5.4339290e-01 -1.6555539e+00 -1.4773605e+00]
#   [-1.0000000e+09 -7.7492166e-01 -6.1755723e-01]]], shape=(2, 2, 3), dtype=float32)

In [ ]:
embeddings.gather(
                        1,
                        torch.cat((state.first_a, current_node), 1)[:, :, None].expand(batch_size, 2, embeddings.size(-1))
                    ).view(batch_size, 1, -1)

In [ ]:
_log_p, pi#tf.expand_dims(pi,axis=-1)

In [ ]:
_log_p

tf.gather(tf.squeeze(_log_p,axis=-2), pi, batch_dims=1)

In [ ]:
_log_p = torch.Tensor(_log_p.numpy())
pi = torch.from_numpy(tf.cast(pi, dtype=tf.int64).numpy()) 

In [ ]:
log_p = _log_p.gather(2, pi.unsqueeze(-1)).squeeze(-1)
log_p.sum(1) 
log_p


In [ ]:
# pi.unsqueeze(-1)
# tf.cast(pi, dtype=tf.int64)
_log_p

In [ ]:
# seq = [torch.Tensor(i.numpy()) for i in seq]
# torch.stack(seq, 1)
pi

In [ ]:
last_node = tf.constant([[0], [1]], dtype=tf.int64)

cur_node = selected[:, tf.newaxis]
cur_node #cur_node

tf.gather_nd(
tf.gather_nd(embedded_inputs.edge_features, tf.concat([last_node, cur_node], axis=1), batch_dims=1),
tf.reshape(tf.range(2), shape=(2,1)), # feature_indexfor processing time
batch_dims=0
)


In [ ]:
A = tf.gather_nd(embedded_inputs.edge_features, tf.concat([last_node, cur_node], axis=1), batch_dims=1)[:,0:1]
A

In [ ]:
tf.reshape(tf.range(2), shape=(2,1))

In [ ]:
embedded_inputs.node_features[:,]

In [ ]:
cur_node[:,tf.newaxis,:] #embedded_inputs.node_features[:, ]
tf.gather_nd(embedded_inputs.edge_features, tf.concat([last_node, cur_node], axis=1), batch_dims=1)[:,0:1] + \
tf.gather_nd(embedded_inputs.node_features, cur_node, batch_dims=1)[:,0:1]

In [ ]:
cur_node

tf.rank(tf.ones(30,10))

t = tf.constant([[[1, 1, 1, 4], [2, 2, 2, 4]], [[3, 3, 3, 4], [4, 4, 4, 4]]])
tf.rank(t)




In [ ]:
_visited = tf.zeros((batch_size,1,3), dtype=tf.uint8)
_visited

In [ ]:
cur_node #[:,:,None] #, tf.ones([2,1,1])

In [ ]:
tf.concat([tf.reshape(tf.range(2, dtype=tf.int64),cur_node.shape), cur_node], axis=1)

In [ ]:
_visited.dtype

In [ ]:
batch_size, _, _ = _visited.shape
tf.tensor_scatter_nd_update( 
    tf.squeeze(_visited, axis=-2),
    tf.concat([tf.reshape(tf.range(batch_size, dtype=tf.int64),cur_node.shape), cur_node], axis=1),
    tf.ones((batch_size,), dtype=_visited.dtype)
)[:,tf.newaxis,:]
    
    
    #_visited, cur_node[:,:,None], tf.ones([2,1,1], dtype=tf.uint8))

In [ ]:
cur_node = selected[:, tf.newaxis]
cur_node #cur_node

tf.concat([cur_node, cur_node], axis=1)

tf.gather_nd(
tf.gather_nd(embedded_inputs.edge_features, tf.concat([cur_node, cur_node], axis=1), batch_dims=1),
tf.zeros((2,1), dtype=tf.int32)
)

tf.gather_nd(embedded_inputs.edge_features, tf.concat([cur_node, cur_node], axis=1), batch_dims=1)

In [ ]:
tf.zeros((2,), dtype=tf.int32)
tf.range(2)

In [ ]:
embedded_inputs.edge_features

In [ ]:
probs_ = tf.squeeze(tf.exp(log_p), axis=-2)
tf.squeeze(mask, axis=-2)

In [ ]:
embedded_inputs.node_features

In [ ]:
probs = torch.Tensor(log_p.numpy()).exp()[:, 0, :]
mask_ = torch.Tensor(mask.numpy())[:, 0, :]

probs, probs.multinomial(1) #.squeeze(1)


# _, selected = probs.max(1)
# assert not mask_.gather(1, selected.unsqueeze(
#                 -1)).data.any(), "Decode greedy: infeasible action has maximum probability"
# mask_.gather(1, selected.unsqueeze(-1)) #.data.any()
probs_

In [ ]:
probs.multinomial(1)

In [ ]:
# p = [.2, .3, .5]
# dist = tfp.distributions.Multinomial(total_count=1, probs=probs_)

p = [[.1, .2, .7], [.3, .3, .4]]  # Shape [2, 3]

dist = tfp.distributions.Multinomial(total_count=1, probs=probs_)
selected = tf.argmax(dist.sample(), axis=1)

In [ ]:
selected = tf.math.argmax(tf.squeeze(tf.exp(log_p), axis=-2), axis=1)

assert not tf.reduce_any(tf.cast(tf.gather_nd(mask_, tf.expand_dims(selected, axis=-1), batch_dims=1), dtype=tf.bool)), "Decode greedy: infeasible action has maximum probability"

In [ ]:
tf.expand_dims(selected, axis=-1), mask_, 

In [ ]:
probs_


In [ ]:
tf.broadcast_to(mask[None, :, None, :, :], shape=compatibility.shape)

In [ ]:
compatibility

compatibility[graph_mask[None, :, :, None, :].expand_as(compatibility)] = -1e10

In [ ]:
def _make_heads(v, num_steps=None):
    assert num_steps is None or v.size(1) == 1 or v.size(1) == num_steps
    n_heads = 2
    return (
        v.contiguous().view(v.size(0), v.size(1), v.size(2), n_heads, -1)
        .expand(v.size(0), v.size(1) if num_steps is None else num_steps, v.size(2), n_heads, -1)
        .permute(3, 0, 1, 2, 4)  # (n_heads, batch_size, num_steps, graph_size, head_dim)
    )

from torch import nn
W_placeholder = nn.Parameter(torch.Tensor(2 * 6))
W_placeholder[None, None, :].expand(2, 1, W_placeholder.size(-1))

In [ ]:
tf.ones((3,4)).dtype

In [ ]:
_make_heads2(A, num_steps=4)

In [ ]:
_make_heads2(A, num_steps=2)

In [ ]:
A.shape
tf.broadcast_to(A, tf.TensorShape([-1, 4, None, None]))

In [ ]:
tf.split(
tf.matmul(tf.ones((5,3))[None,:,:], tf.ones((3,3*3))),
3,
axis=-1

)

In [ ]:
tf.matmul(tf.ones((5,3))[None,:,:], tf.ones((3,3*3)))

In [ ]:
import torch
embeddings = torch.Tensor(embedded_inputs.node_features.numpy())

torch.cat((state.first_a, current_node), 1)[:, :, None]

In [ ]:
embedded_inputs.node_features

In [ ]:
import torch
from torch import nn

W_placeholder = nn.Parameter(torch.Tensor(2 * 5))


W_placeholder[None, None, :].expand(batch_size, 1, W_placeholder.size(-1)).shape

In [ ]:
tf.constant(W_placeholder[None, None, :].detach().numpy()).shape[-1]

In [ ]:
tf.broadcast_to?

In [ ]:
W_placeholder[None, None, :].shape

In [ ]:
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)

In [ ]:
tf.reduce_mean(embedded_inputs.node_features, axis=-2)

In [ ]:
import torch

In [ ]:
torch.Tensor(embedded_inputs.node_features.numpy()).max(1)[0]

In [ ]:
embedded_inputs.node_features.numpy()

In [ ]:
"""

"""
import tensorflow as tf
from tensorflow.keras import Model

from msp.layers import GGCNLayer
from msp.graphs import MSPSparseGraph
from msp.solutions import MSPState

class AttentionDecoder(Model):

    def __init__(self, 
                 units,
                 *args, 
                 activation='relu',
                 aggregation_graph='mean',
                 n_heads=8,
                 **kwargs):
        """ """
        super(AttentionDecoder, self).__init__(*args, **kwargs)
        self.aggregation_graph = aggregation_graph
        self.n_heads = n_heads

        embedding_dim = units
        self.W_placeholder = self.add_weight(shape=(2*embedding_dim,),
                                initializer='random_uniform', #Placeholder should be in range of activations (think)
                                trainable=True)

        graph_embed_shape = tf.TensorShape((None, units))
        self.fixed_context_layer = tf.keras.layers.Dense(units, use_bias=False)
        self.fixed_context_layer.build(graph_embed_shape)

        # For each node we compute (glimpse key, glimpse value, logit key) so 3 * embedding_dim
        project_node_embeddings_shape = tf.TensorShape((None, None, None, units))
        self.project_node_embeddings = tf.keras.layers.Dense(3*units, use_bias=False)
        self.project_node_embeddings.build(project_node_embeddings_shape)

        #
        # Embedding of first and last node
        step_context_dim = 2*units
        project_step_context_shape = tf.TensorShape((None, None, step_context_dim))
        self.project_step_context = tf.keras.layers.Dense(embedding_dim, use_bias=False)
        self.project_step_context.build(project_step_context_shape)
        
        #nn.Linear(step_context_dim, embedding_dim, bias=False)




        # self.context_layer = tf.keras.layers.Dense(units, use_bias=False)
        # self.mha_layer = None
        

        # dynamic router


        


        # self.initial_layer_1 = tf.keras.layers.Dense(units)
        # self.initial_layer_2 = tf.keras.layers.Dense(units)
        # self.ggcn_layers = [GGCNLayer(units, activation=activation)
        #                     for _ in range(layers)]
        
    def call(self, inputs, training=False):
        """ """
        state = MSPState(inputs)

        node_embedding = inputs.node_embed

        # AttentionModelFixed(node_embedding, fixed_context, *fixed_attention_node_data)
        fixed = self._precompute(node_embedding)
        return fixed 

        while not state.all_finished():

            log_p, mask = self._get_log_p(fixed, state)

        

        node_embed = inputs.node_features
        return self._get_parallel_step_context(node_embed)


    def _precompute(self, node_embedding, num_steps=1):

        graph_embed = self._get_graph_embed(node_embedding)

        fixed_context = self.fixed_context_layer(graph_embed)
        # fixed context = (batch_size, 1, embed_dim) to make broadcastable with parallel timesteps
        fixed_context = tf.expand_dims(fixed_context, axis=-2)

        glimpse_key_fixed, glimpse_val_fixed, logit_key_fixed  = tf.split(
            self.project_node_embeddings(tf.expand_dims(node_embedding, axis=-3)),
            num_or_size_splits=3,
            axis=-1
        )

        return glimpse_key_fixed

        # No need to rearrange key for logit as there is a single head
        fixed_attention_node_data = (
            self._make_heads(glimpse_key_fixed, num_steps),
            self._make_heads(glimpse_val_fixed, num_steps),
            logit_key_fixed
        )
        return AttentionModelFixed(node_embedding, fixed_context, *fixed_attention_node_data)

    def _get_graph_embed(self, node_embedding):
        """ """
        if self.aggregation_graph == "sum":
            graph_embed = tf.reduce_sum(node_embedding, axis=-2)
        elif self.aggregation_graph == "max":
            graph_embed = tf.reduce_max(node_embedding, axis=-2)
        elif self.aggregation_graph == "mean":
            graph_embed = tf.reduce_mean(node_embedding, axis=-2)
        else:  # Default: dissable graph embedding
            graph_embed = tf.reduce_sum(node_embedding, axis=-2) * 0.0

        return graph_embed

    def _make_heads(self, v, num_steps=None):

        assert num_steps is None or v.shape[1] == 1 or v.shape[1] == num_steps
        
        batch_size, _, num_nodes, embed_dims = v.shape
        num_steps = num_steps if num_steps else 1

        # M x B x N x V x H
        return tf.broadcast_to(
            tf.broadcast_to(v, shape=[batch_size, num_steps, num_nodes, embed_dims])[None,:,:,:,:],
            shape=[self.n_heads, batch_size, num_steps, num_nodes, embed_dims]
        )

    def _get_log_p(self, fixed, state, normalize=True):
        # Compute query = context node embedding
        
        # B x 1 x H
        query = fixed.context_node_projected + \
                self.project_step_context(self._get_parallel_step_context(fixed.node_embeddings, state))
        
        # Compute keys and values for the nodes
        glimpse_K, glimpse_V, logit_K = self._get_attention_node_data(fixed, state)

        graph_mask = None
        if self.mask_graph:
            # Compute the graph mask, for masking next action based on graph structure 
            graph_mask = state.get_graph_mask()  # Pending...........................................

        # Compute logits (unnormalized log_p)
        log_p, glimpse = self._one_to_many_logits(query, glimpse_K, glimpse_V, logit_K, mask, graph_mask)
        


    def _get_parallel_step_context(self, node_embedding, state, from_depot=False):
        """
        Returns the context per step, optionally for multiple steps at once 
        (for efficient evaluation of the model)
        """
        current_node = state.get_current_node()
        batch_size, num_steps = current_node.shape

        if num_steps == 1:  # We need to special case if we have only 1 step, may be the first or not
            if self.i.numpy()[0] == 0:
                # First and only step, ignore prev_a (this is a placeholder)
                # B x 1 x 2H
                return tf.broadcast_to(self.W_placeholder[None, None, :], 
                                       shape=[batch_size, 1, self.W_placeholder.shape[-1]])
                
            # else:
            #     return embeddings.gather(
            #         1,
            #         torch.cat((state.first_a, current_node), 1)[:, :, None].expand(batch_size, 2, embeddings.size(-1))
            #     ).view(batch_size, 1, -1)

    def _get_attention_node_data(self, fixed, state):
        return fixed.glimpse_key, fixed.glimpse_val, fixed.logit_key

    def _one_to_many_logits(self, query, glimpse_K, glimpse_V, logit_K, mask, graph_mask=None):
        pass




    

In [ ]:
tf.boolean_mask

In [ ]:
MSPSparseGraph._fields

In [ ]:
dataset = make_sparse_data(4, msp_size=(1,2), random_state=2021)

In [ ]:
graph = list(dataset.take(1))[0]
graph

In [ ]:
indices= tf.cast(tf.transpose(
    tf.concat([
        edge_index,
        tf.scatter_nd(
            tf.constant([[1],[0]]),
            edge_index,
            edge_index.shape
        )
    ], axis=-1)
), tf.int64)
indices

In [ ]:
adj_matrix = tf.sparse.SparseTensor(
    indices= indices,
    values = tf.ones([2*edge_index.shape[-1]]),
    dense_shape = [3,3]
)
adj_matrix

In [ ]:
tf.sparse.to_dense(tf.sparse.reorder(adj_matrix))

In [ ]:
reorder

In [ ]:
tf.ones([edge_index.shape[-1]])

In [ ]:
E = tf.sparse.SparseTensor(indices=[[0, 0], [1, 2], [2, 0]], values=[1, 2], dense_shape=[3, 4])
tf.sparse.to_dense()

In [ ]:
tf.ones((2,3))

In [ ]:
class GGCNLayer(Layer):

    # @validate_hyperparams
    def __init__(self, 
                 units,
                 *args, 
                 activation='relu', 
                 use_bias=True, 
                 normalization='batch',
                 aggregation='mean',
                 **kwargs):
        """ """
        super(GGCNLayer, self).__init__(*args, **kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        self.use_bias = use_bias
        self.normalization= normalization
        self.aggregation = aggregation

    def build(self, input_shape):
        """Create the state of the layer (weights)"""
        print('Build')
        node_features_shape = input_shape.node_features
        edge_featues_shape = input_shape.edge_features
        embedded_shape = tf.TensorShape((None, self.units))

        # _initial_projection_layer (think on it)

        with tf.name_scope('node'):
            with tf.name_scope('U'):
                self.U = tf.keras.layers.Dense(self.units, use_bias=self.use_bias)
                self.U.build(node_features_shape)

            with tf.name_scope('V'):
                self.V = tf.keras.layers.Dense(self.units, use_bias=self.use_bias)
                self.V.build(node_features_shape)

            with tf.name_scope('norm'):
                self.norm_h = {
                    "batch": tf.keras.layers.BatchNormalization(),
                    "layer": tf.keras.layers.LayerNormalization()
                }.get(self.normalization, None)
                if self.norm_h:
                    self.norm_h.build(embedded_shape)

        with tf.name_scope('edge'):
            with tf.name_scope('A'):
                self.A = tf.keras.layers.Dense(self.units, use_bias=self.use_bias)
                self.A.build(tf.TensorShape((None, node_features_shape[-1])))
            
            with tf.name_scope('B'):
                self.B = tf.keras.layers.Dense(self.units, use_bias=self.use_bias)
                self.B.build(node_features_shape)

            with tf.name_scope('C'):
                self.C = tf.keras.layers.Dense(self.units, use_bias=self.use_bias)
                self.C.build(edge_featues_shape)

            with tf.name_scope('norm'):
                self.norm_e = {
                    'batch': tf.keras.layers.BatchNormalization(),
                    'layer': tf.keras.layers.LayerNormalization(axis=-2)
                }.get(self.normalization, None)
                if self.norm_e:
                    self.norm_e.build(embedded_shape)
    
        super().build(input_shape)
 
    def call(self, inputs):
        """ """
        print('call')
        adj_matrix = inputs.adj_matrix
        h = inputs.node_features
        e = inputs.edge_features

        # Edges Featuers
        Ah = self.A(h)
        Bh = self.B(h)
        Ce = self.C(e)
        e = self._update_edges(e, [Ah, Bh, Ce])

        edge_gates = tf.sigmoid(e)

        # Nodes Features
        Uh = self.U(h)
        Vh = self.V(h)
        h = self._update_nodes(
            h,
            [Uh, self._aggregate(Vh, edge_gates, adj_matrix)]
        )

        outputs = MSPSparseGraph(adj_matrix, h, e, inputs.alpha)
        return inputs
        
    def _update_edges(self, e, transformations:list):
        """Update edges features"""
        Ah, Bh, Ce  = transformations
        e_new = tf.expand_dims(Ah, axis=1) + tf.expand_dims(Bh, axis=2) + Ce
        # Normalization
        batch_size, num_nodes, num_nodes, hidden_dim = e_new.shape
        if self.norm_e:
            e_new = tf.reshape(
                self.norm_e(
                    tf.reshape(e_new, [batch_size*num_nodes*num_nodes, hidden_dim])
                ), e_new.shape
            )
        # Activation
        e_new = self.activation(e_new)
        # Skip/residual Connection
        e_new = e + e_new
        return e_new

    def _update_nodes(self, h, transformations:list):
        """ """
        Uh, aggregated_messages = transformations
        h_new = tf.math.add_n([Uh, aggregated_messages])
        # Normalization
        batch_size, num_nodes, hidden_dim = h_new.shape
        if self.norm_h:
            h_new = tf.reshape(
                self.norm_h(
                    tf.reshape(h_new, [batch_size*num_nodes, hidden_dim])
                ), h_new.shape
            )
        # Activation
        h_new = self.activation(h_new)
        # Skip/residual Connection
        h_new = h + h_new
        return h_new

    def _aggregate(self, Vh, edge_gates, adj_matrix):
        """ """
        # Reshape as edge_gates
        Vh = tf.broadcast_to(
            tf.expand_dims(Vh, axis=1),
            edge_gates.shape
        )
        # Gating mechanism
        Vh = edge_gates * Vh
        
        # Enforce graph structure
        # mask = tf.broadcast_to(tf.expand_dims(adj_matrix,axis=-1), Vh.shape)
        # Vh[~mask] = 0

        # message aggregation
        if self.aggregation == 'mean':
            total_messages = tf.cast(
                tf.expand_dims(
                    tf.math.reduce_sum(adj_matrix, axis=-1),
                    axis=-1
                ),
                Vh.dtype
            )
            return tf.math.reduce_sum(Vh, axis=2) / total_messages
        
        elif self.aggregation == 'sum':
            return tf.math.reduce_sum(Vh, axis=2)


In [ ]:
#

In [ ]:

B, V, H = 1, 3, 2
h = tf.ones((B, V, H))


In [ ]:
dataset = make_sparse_data(4, msp_size=(1,2), random_state=2020)
graphs = list(dataset.batch(2))
VVh = list(graphs)[0].edge_features
A = list(graphs)[0].adj_matrix
VVh.shape, A.shape

In [ ]:
# mask = tf.cast(tf.broadcast_to(
#     tf.expand_dims(A, axis=-1),
#     VVh.shape
# ), tf.bool)
# VVh[~mask] 
ggcn.variables

In [ ]:
units = graphs[0].edge_features.shape[-1]
ggcn = GGCNLayer(units=units, use_bias=True, name='GGCN_Layer')
output = ggcn(graphs[0])
output

In [ ]:
tf.cast(
    tf.expand_dims(
        tf.math.reduce_sum(list(graphs)[0].adj_matrix, axis=-1),
        axis=-1
    ),
    Vh.dtype
)
    tf.expand_dims(
    tf.math.reduce_sum(list(graphs)[0].adj_matrix, axis=-1),
    axis=-1).dtype

In [ ]:
tf.math.reduce_max(graphs[0].edge_features, axis=-2 )[0]

In [ ]:
list(graphs)[0].adj_matrix

In [ ]:
# ggcn = GGCNLayer(units=units, use_bias=True, name='GGCN_Layer')
# output = ggcn(graphs[0])
# output

In [ ]:
graphs = list(make_data(1, msp_size=(1,2), random_state=2021).take(1))[0]

num_edges = 3


A = tf.sparse.SparseTensor(
    indices= tf.cast(tf.transpose(graphs['edge_index']), tf.int64),
    values = tf.ones([num_edges]),
    dense_shape = [3,3]
)

tf.sparse.to_dense(A)

In [ ]:
tf.cast(tf.transpose(graphs['edge_index']), tf.int64)

In [ ]:
# ggcn.variables
tf.cast(tf.transpose(graphs['edge_index']), tf.int64)


In [ ]:
edge_index = tf.cast(tf.transpose(
    tf.concat([
        graphs['edge_index'],
        tf.scatter_nd(
            tf.constant([[1],[0]]),
            graphs['edge_index'],
            graphs['edge_index'].shape
        )
    ], axis=-1)
), tf.int32)
edge_index

In [ ]:
model = tf.keras.Sequential([
    GGCNLayer(units=units, use_bias=True, name='GGCN_Layer')
])
model.compile(optimizer='sgd', loss='mse')
model.fit(dataset)

In [ ]:
model.weights